In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

# define torrent to download from
torrent_file_url = "https://academictorrents.com/download/af04abfe9a3c96b30e5dd029eb185e19a7055272.torrent"

# define destination directory
directory_path = "/content/drive/MyDrive/UIUC MCS/CS598 Deep Learning for Healthcare/Project/data/"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install libtorrent requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 64.5 MB/s eta 0:00:00


In [ ]:
import os
import time
import requests
import libtorrent as lt

def download_torrent(torrent_file_url, save_path):
    ses = lt.session()
    ses.listen_on(6881, 6891)

    # Checkpoint file path
    checkpoint_path = save_path + 'session_checkpoint.dat'

    # Load session state if exists
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            ses.load_state(lt.bdecode(f.read()))
        print("Session state loaded from checkpoint.")

    r = requests.get(torrent_file_url)
    torrent_file_path = save_path + 'temp.torrent'
    with open(torrent_file_path, 'wb') as f:
        f.write(r.content)

    params = {
        'save_path': save_path,
        'storage_mode': lt.storage_mode_t.storage_mode_sparse,
        'paused': False,
        'auto_managed': True,
        'duplicate_is_error': True
    }

    info = lt.torrent_info(torrent_file_path)
    h = ses.add_torrent({'ti': info, 'save_path': save_path})
    print('Downloading metadata...')
    while not h.has_metadata():
        time.sleep(1)
    print('Got metadata, starting torrent download...')

    while (h.status().state != lt.torrent_status.seeding):
        s = h.status()
        print('%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' % (
            s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
            s.num_peers, s.state))

        # Save session state periodically
        with open(checkpoint_path, 'wb') as f:
            f.write(lt.bencode(ses.save_state()))
        time.sleep(5)

    print("Download completed.")
    # Optionally, remove the checkpoint file after download completion
    # os.remove(checkpoint_path)
    # os.remove(torrent_file_path)

download_torrent(torrent_file_url, directory_path)


<ipython-input-2-a0e075dd1ea4>:8: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)


Got metadata, starting torrent download...
0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 0) checking_resume_data


<ipython-input-2-a0e075dd1ea4>:35: DeprecationWarning: has_metadata() is deprecated
  while not h.has_metadata():


0.00% complete (down: 1115.2 kB/s up: 30.3 kB/s peers: 1) downloading
0.00% complete (down: 926.8 kB/s up: 25.7 kB/s peers: 4) downloading
0.01% complete (down: 4251.0 kB/s up: 127.2 kB/s peers: 4) downloading
0.02% complete (down: 4129.6 kB/s up: 122.9 kB/s peers: 4) downloading
0.04% complete (down: 11007.2 kB/s up: 313.3 kB/s peers: 4) downloading
0.07% complete (down: 13645.0 kB/s up: 384.8 kB/s peers: 4) downloading
0.10% complete (down: 14937.2 kB/s up: 420.4 kB/s peers: 4) downloading
0.13% complete (down: 16788.9 kB/s up: 471.1 kB/s peers: 4) downloading
0.17% complete (down: 21587.4 kB/s up: 600.0 kB/s peers: 4) downloading
0.21% complete (down: 21880.8 kB/s up: 606.6 kB/s peers: 4) downloading
0.24% complete (down: 18847.2 kB/s up: 525.5 kB/s peers: 4) downloading
0.27% complete (down: 16825.8 kB/s up: 471.2 kB/s peers: 4) downloading
0.30% complete (down: 16963.5 kB/s up: 473.8 kB/s peers: 4) downloading
0.33% complete (down: 14577.3 kB/s up: 408.1 kB/s peers: 4) downloading